In [2]:
from langchain.document_loaders import WikipediaLoader

In [3]:
loader = WikipediaLoader(query="MKUltra")
documents = loader.load()

/Users/pete/.local/share/virtualenvs/langchain-udemy-notebooks-Raz3oF7P/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/pete/.local/share/virtualenvs/langchain-udemy-notebooks-Raz3oF7P/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [4]:
len(documents)

9

In [5]:
from langchain.text_splitter import CharacterTextSplitter

In [7]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents=documents)

In [8]:
len(docs)

19

In [9]:
from langchain.embeddings import OpenAIEmbeddings

In [10]:
embedding_function = OpenAIEmbeddings()

In [11]:
from langchain.vectorstores import Chroma

In [12]:
db = Chroma.from_documents(docs, embedding_function, persist_directory="./mk-ultra")

In [13]:
db.persist()

The multi query

In [14]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chat_models import ChatOpenAI

In [15]:
question = "when was this declassified?"

In [16]:
llm = ChatOpenAI(temperature=0)

In [17]:
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=db.as_retriever(), llm=llm)

In [18]:
# Logging behind scenes
import logging
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

In [19]:
# This will not directly answer the query, just get n relevant documents
unique_docs = retriever_from_llm.get_relevant_documents(query=question)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the date of declassification for this information?', '2. Can you provide the declassification date for this?', '3. When was the document in question officially declassified?']


In [20]:
len(unique_docs)

5